<center>

## MAGICIAN: Mining and gAmblinG servIces sCrapIng and ANalysis

Matteo Martini 636694

In [1]:
mining_pool_url = "https://en.bitcoin.it/wiki/DeepBit9"


#Individuare nel DataSet tutte le transazioni relative a Deepbit.net,ovvero le transazioni che presentano almeno un indirizzo di input o di output appartenente a Deepbit_addrs.
#servizio scommesse = DiceOnCrack

## Setup Ambiente

In [1]:
# Librerie concesse
import pandas as pd

# Per scraping (scegline uno o entrambi, in base alle esigenze delle pagine)
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

# Per formati efficienti (PyArrow)
import pyarrow as pa
import pyarrow.parquet as pq

# Utility
from pathlib import Path
from datetime import datetime

import numpy as np


from selenium.webdriver.support.select import Select
from selenium.webdriver.common.keys import Keys
import time
import requests
import re
from bs4 import BeautifulSoup

# Percorsi (aggiorna con il tuo percorso locale)
DATA_DIR = Path('./Datasets')   # metti qui i CSV scaricati

PARQUET_DIR = Path('./data_parquet')
PARQUET_DIR.mkdir(parents=True, exist_ok=True)
print(DATA_DIR)
print(PARQUET_DIR)
print('Environment ready.')

Datasets
data_parquet
Environment ready.


## 1) Costruzione dei Dataframe dal Dataset

## Directory Datasets


In [2]:
transactions_csv = DATA_DIR / 'transactions.csv'
inputs_csv       = DATA_DIR / 'inputs.csv'
outputs_csv      = DATA_DIR / 'outputs.csv'
mapping_csv      = DATA_DIR / 'mapAddr2Ids8708820.csv'

## Definizone Data Type
Sapendo cosa contengono i file csv, possiamo forzare la conversione in determiinati tipi per ridurre l'impatto in memoria dei dataset

In [3]:
dtype_transactions = {
    "timestamp": np.int32,          # prima colonna: secondi (1231469xxx) rientra in int32
    "tx_id": np.int32,              # seconda colonna: contatore 1…10 572 827
    "block_height": np.int32,       # terza colonna: 1…214 562
    "prev_tx_id": np.int32,         # quarta colonna: seguono il tx_id precedente
    "has_witness": np.uint8,        # quinta colonna 0/1
    "fee": np.int32,                # sesta colonna: 0…100 000 (int32 più che sufficiente)
}


dtype_inputs = {
    "input_id": np.int32,
    "tx_id": np.int32,
    "prev_output_id": np.int32,
    "input_index": np.uint8,
}


dtype_outputs = {
    "col0":np.int32,
    "col1":np.int32,
    "col2":np.int8,
    "col3":np.int32,
    "col4":np.int64,
    "col5":"uint8"
}

dtype_mapping = {
    "address": "category",   # stringhe ripetute → category riduce drasticamente
    "address_id": "Int32",   # valori 0–8 708 820 entrano in 32 bit
}

columns_transactions = [
    "timestamp", "tx_id", "block_height",
     "has_witness", "fee"
]

## Caricamento dei Dataset

In [43]:
transactions_csv = DATA_DIR / 'transactions.csv'
inputs_csv       = DATA_DIR / 'inputs.csv'
outputs_csv      = DATA_DIR / 'outputs.csv'
mapping_csv      = DATA_DIR / 'mapAddr2Ids8708820.csv'

transactions = pd.read_csv(transactions_csv,header=None,names=columns_transactions,dtype=dtype_transactions)
inputs_df    = pd.read_csv(inputs_csv,dtype=dtype_inputs)
outputs_df   = pd.read_csv(outputs_csv,dtype=dtype_outputs)
mapping_df   = pd.read_csv(mapping_csv,dtype=dtype_mapping)

## 2) Recupero informazioni da WalletExplorer


### Definizione Funzioni Helper per Scraping

In [4]:

#placeholder
wallet_explorer_url = "https://www.walletexplorer.com/"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/127.0.0.1 Safari/537.36"
    ),
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.imdb.com/",
}

In [5]:
def get_transactions_link(soup,class_):
    transactions = soup.find_all('tr',class_=class_)
    page_links = []
    for trans in transactions:
        txid = trans.find('td',class_="txid").find('a')['href']
        page_links.append(txid)
    return page_links

def get_page_transactions(url):
    res = requests.get(f'{url}',headers=headers)
    soup = BeautifulSoup(res.text,"html.parser")
    global_trans = get_transactions_link(soup,'sent') + get_transactions_link(soup,'received')
    return global_trans


### Definizione Funzione per Scraping

In [6]:
def next_page(driver):
    #cerco il link alla prossima pagina
    for _ in ("Next","Last"):
        url = find_next_link(driver,_)
        if url:return url
    return None
        
def find_next_link(driver,element):
    try:
        el = driver.find_element(By.XPATH, f'//div[@class="paging"]//a[contains(., "{element}")]')
        url = el.get_attribute("href")
        el.click()
        return url
    except:
        return None

def scrape_step(driver,url):
    #recupero le entries
    page_entries = get_page_transactions(url)
    #vado alla prossima pagina
    url = next_page(driver)
    return [url,page_entries]

def scrape_wallet_explorer(wallet_name):
    driver = webdriver.Chrome()
    driver.get(wallet_explorer_url)
    
    #ricerca del wallet
    driver.find_element(By.NAME,'q').send_keys(wallet_name)
    driver.find_element(By.TAG_NAME,'form').submit()

    url = driver.current_url
    i = 0
    all_entries = []
    while True:
        #recupero di tutte le transazioni della pagina
        url,page_entries = scrape_step(driver,url)
        all_entries.extend(page_entries)
        if url == None:
            break;
    driver.close()
    return all_entries

#https://www.walletexplorer.com/txid/d71d3b18df328681c4e0e4ed215a19820c7ecff956e1b6961eec10b7b3417917


In [7]:
def get_info_from_transaction_table(trans_table_link):
    res = requests.get(f'{"https://www.walletexplorer.com"}{trans_table_link}',headers=headers)
    soup = BeautifulSoup(res.text,"html.parser")
    table = soup.find('table',class_="info").find_all('td')
    all_txt = []
    for element in table:
        txt = element.get_text()
        all_txt.append(txt)
    
    return {
        "timestamp":all_txt[2],
        "blockId":int(re.search(r'\d+', all_txt[1]).group()),
        "txId":all_txt[0],
        "isCoinbase":0,
        "fee":float(re.search(r'\d+(?:\.\d+)?', all_txt[4]).group())
    }

In [8]:
deepbit_transaction = scrape_wallet_explorer('DiceonCrack')
#1.2296645641326904
#0.6821258068084717
#BW.com

In [116]:
# print(deepbit_transaction[len(deepbit_transaction)-1])
info = get_info_from_transaction_table(deepbit_transaction[len(deepbit_transaction)-1])
print(f'{info["fee"]}')

0.0001


In [9]:
def get_all_trans(transaction_list):
    infos = []
    for transaction_link in transaction_list:
        info = get_info_from_transaction_table(transaction_link)
        infos.append(info)
    return infos

In [ ]:
doc_trans = get_all_trans(deepbit_transaction)

KeyboardInterrupt: 